In [1]:
#Import the required modules
from dolfin import *
import numpy as np

/usr/lib/pymodules/python2.7/petsc4py/lib/__init__.py:123: UserWarning: ignored arch: 'arch-linux2-cxx-debug', using: 'linux-gnu-c-opt'
  (os.path.pathsep.join(arch_env), arch))


In [2]:
#The mesh used for the rectangular hollow guides
a = 1.0
b = 0.5


In [3]:
#Create a rectancular mesh with origin (0,0) extending to (a,b) with 8 edges along the long side and 4 elements along the short side
mesh =RectangleMesh(Point(0,0),Point(a,b),8,4)

In [4]:
#define the orders of the fucntion spaces for vector and nodal basis functions
vector_order = 2
nodal_order = 3


In [5]:
#define the function spaces
vector_space = FunctionSpace(mesh,"Nedelec 1st kind H(curl)",vector_order)
nodal_space = FunctionSpace(mesh,"Lagrange",nodal_order)
combined_space = vector_space*nodal_space

DEBUG:FFC:Reusing form from cache.
DEBUG:FFC:Reusing form from cache.
DEBUG:FFC:Reusing form from cache.


In [6]:
#Define the test and trial functions from the combined space
# here N_v and N_u are Nedelec basis functions and L_v and L_u are Lagrange basis functions
(N_i,L_i) = TestFunctions(combined_space)
(N_j,L_j) = TrialFunctions(combined_space)

In [7]:
#specify the relative permittivity and permeability
e_r = 1.0
u_r = 1.0

In [8]:
def curl_t(w):
    return Dx(w[1], 0) - Dx(w[0], 1)

In [9]:
#define the forms (matrix elements) for cutoff analysis into the basis fnctions
s_tt_ij = 1.0/u_r*inner(curl(N_i),curl(N_j))
t_tt_ij = e_r*inner(N_i,N_j)
s_zz_ij = 1.0/u_r * inner(grad(L_i),grad(L_j))
t_zz_ij = e_r*L_i*L_j
#post-multiplication by dx will result in integration over the domain of the mesh at assembly time
s_ij =(s_tt_ij+s_zz_ij)*dx
t_ij = (t_tt_ij+t_zz_ij)*dx

In [10]:
#assemble the sysstem Matrices. DOLFIN automtically evaluates each of the forms for all the 
#relavent test and tial functions combnatuoins ie piossible valies of i and j
S = assemble(s_ij)
T = assemble(t_ij)


DEBUG:FFC:Reusing form from cache.
DEBUG:FFC:Reusing form from cache.


In [11]:
#create a mesh function to mark the edges(dimension 1) in the mesh
boundary_markers = MeshFunction('uint',mesh,1)
#mark all edges as 0
boundary_markers.set_all(0)
#mark the edges on the boundary as 1
DomainBoundary().mark(boundary_markers,1)

In [12]:
#create the boundary condition using the combined function space, a zero Expression, and the mesh function
#for the edges. Note that the last parameter (1) is used to indicate the edges where the boundary condition must be applied
electric_wall = DirichletBC(combined_space,Expression(("0.0","0.0","0.0")),boundary_markers,1)
#apply the boundary condition to the assembled matrices
electric_wall.apply(S)
electric_wall.apply(T)

In [13]:
#initialise a vector of ones
indicators = np.ones(S.size(0))
#get the boundary indictators to remove the rows and colums associated with the boundary DOFs
indicators[electric_wall.get_boundary_values().keys()]=0
#the free DOFs correspond to the elements of indicators that are equal to 1
free_dofs = np.where(indicators == 1)[0]
#convert the dolfin matrices to numpy arrays selecting only the rows and colums associated with the free DOFs
S_np = S.array()[free_dofs,:][:,free_dofs]
T_np = T.array()[free_dofs,:][:,free_dofs]

In [14]:
#solve the eigensystem (S_np,T_np) using scipy)]
from scipy.linalg import eig
k_c_squared,ev = eig(S_np,T_np)
#sort the calculated values
sort_index = np.argsort(k_c_squared)
#skip over the non-physical(zero) eigenmodes
first_mode_idx = np.where(k_c_squared[sort_index]>1e-8)[0][0]

In [15]:
print("The cutoff wavenumbers of the 4 most dominant modes are:")
print(k_c_squared[sort_index][first_mode_idx:first_mode_idx+4])

The cutoff wavenumbers of the 4 most dominant modes are:
[  9.86961866+0.j  39.47928042+0.j  39.47928196+0.j  49.34837428+0.j]


In [16]:
#TE_10 mode is the first mode
mode_idx = 0
#post-process the coefficients to map back to the full matrix
coefficiants_global = np.zeros(S.size(0))
coefficiants_global[free_dofs] = ev[:,sort_index[first_mode_idx+mode_idx]]
#Create a Function on the combined space
mode = Function(combined_space)
#Assign the coefficients of the function to the calculated values
mode.vector().set_local(coefficiants_global)
#Split the function into the parts in each of the functions spaces in combined_space
#This is done using DOLFINs Function.split()
(TE,TM) = mode.split()
#Plot the mode using the dolfin plotter


In [17]:
plot(TE,title = "TE_10 mode")
interactive()

In [18]:
coefficiants_global = np.zeros(S.size(0))
coefficiants_global[free_dofs] = ev[:,sort_index[first_mode_idx+mode_idx]]

In [57]:
#TE_10 mode is the first mode
mode_idx = 3
#post-process the coefficients to map back to the full matrix
coefficiants_global = np.zeros(S.size(0))
coefficiants_global[free_dofs] = ev[:,sort_index[first_mode_idx+mode_idx]]
#Create a Function on the combined space
mode = Function(combined_space)
#Assign the coefficients of the function to the calculated values
mode.vector().set_local(coefficiants_global)
#Split the function into the parts in each of the functions spaces in combined_space
#This is done using DOLFINs Function.split()
(TE,TM) = mode.split()
#Plot the mode using the dolfin plotter

In [58]:
plot(TM,title="TM_11 mode")
interactive()

In [21]:
ev

array([[  2.42890614e-03,  -4.26559435e-03,   2.90552536e-03, ...,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
       [  1.34922843e-03,  -4.40257309e-03,   4.94730813e-04, ...,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
       [ -0.00000000e+00,  -0.00000000e+00,  -0.00000000e+00, ...,
          3.91102663e-01,   8.35385806e-02,  -8.45602941e-03],
       ..., 
       [ -2.08628507e-06,   3.54715095e-05,   2.53059122e-05, ...,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
       [  2.90403488e-05,   6.67313327e-05,  -2.47572575e-05, ...,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
       [ -0.00000000e+00,  -0.00000000e+00,  -0.00000000e+00, ...,
         -1.94603331e-01,  -4.26800459e-01,   2.52950246e-01]])

In [22]:
S.size(0)

669

In [55]:
size1 = 100
size2 = 100
E,E_axial = electric_field(mode_idx,size1,size2,k,S,free_dofs,ev,first_mode_idx)

In [56]:
mode_field1 = np.transpose((np.abs(E[:,:,0])**2 + np.abs(E[:,:,1])**2+np.abs(E_axial[:,:])**2)**0.5)
maxi = np.max(mode_field1)
mode_field1 /=maxi
x = np.linspace(0,a,size1)
y = np.linspace(0,b,size2)
X,Y = np.meshgrid(x,y)
plt.contourf(X,Y,mode_field1,90)
plt.colorbar()
#plt.title(r'mode$=$'+str(mode_idx)+', '+' Effective=_index $=$'+str(beta[sort_index][mode_idx]/k0))
plt.show()

In [26]:
import matplotlib.pylab as plt

In [ ]:
plot(TM)